In [1]:
import os
import numpy as np
import pandas as pd
import itertools
import skimage.io as skio
import skimage.measure as skim
import skimage.transform as skit

import pulse2percept as p2p
import p2pspatial

%matplotlib inline
import matplotlib.pyplot as plt

2018-01-15 15:31:24,583 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
subject = '12-005'
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')
X, y = p2pspatial.load_data(rootfolder, subject=subject, amplitude=2.0)
electrodes = X.electrode.unique()
electrodes

array(['A06', 'C10', 'D04', 'D07', 'E01', 'E03', 'E07', 'E10', 'F02',
       'F04', 'F06', 'F09', 'A01', 'A04', 'A08', 'B01', 'B03', 'B09',
       'C07', 'D01', 'D08', 'D10', 'C04', 'C08', 'A10'], dtype=object)

In [ ]:
for comb in itertools.combinations(y.iloc[:4, :].iterrows(), 2):
    print(comb[0][0], comb[1][0], p2pspatial.imgproc.dice_loss(comb, w_deg=0.1, n_angles=41))

0 1 9.98831158084
0 2 2.97596996607
0 3 1.93965408631
1 2 12.4344689159
1 3 11.5144550693
2 3 0.872602869298


In [ ]:
losses = []
labels = []
for electrode in electrodes:
    print('Processing %s' % electrode)
    X, y = p2pspatial.load_data(rootfolder, subject=subject, 
                                electrodes=electrode, amplitude=2.0)
    losses.append(p2p.utils.parfor(p2pspatial.imgproc.dice_loss, 
                                   itertools.combinations(y.iterrows(), 2), 
                                   func_kwargs={'w_deg': 0.1, 'n_angles': 41}))
    labels.append(electrode)

Processing A06
Processing C10


In [ ]:
plt.boxplot(losses, labels=labels)
plt.title(subject)